In [1]:
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from sklearn import preprocessing
import pandas as pd
import numpy as np
import glob, hdbscan

import operator

from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.sparse.csgraph import connected_components

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
import random;

from pyclustering.cluster import cluster_visualizer;
from pyclustering.cluster.optics import optics, ordering_analyser, ordering_visualizer;

from pyclustering.utils import read_sample, timedcall;


In [16]:
??read_sample

In [3]:
??optics

In [6]:
train = np.unique([p.split('-')[0] for p in sorted(glob.glob('../data/train/**'))])
test = np.unique([p.split('-')[0] for p in sorted(glob.glob('../data/test/**'))])

In [7]:
scl = preprocessing.StandardScaler()

#https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
#https://www.kaggle.com/mikhailhushchyn/hough-transform
def norm_points(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    df['x2'] = x/r
    df['y2'] = y/r
    df['z2'] = z / np.sqrt(x**2 + y**2)
        
    return df

def norm_points2(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    df['x2'] = x/r
    df['y2'] = y/r

#     r = np.sqrt(x**2 + y**2)
    df['z2'] = z/r
    
    return df

def norm_points3(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values

    r = np.sqrt(x**2 + y**2)
    df['x2'] = r
#     df['y2'] = y/r

#     r = np.sqrt(x**2 + y**2)
    df['z2'] = z
    
    return df

In [17]:
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    hits = norm_points(hits)
    print(hits.head())
    
    
    hits['phi'] = np.degrees(np.arctan2(hits['y2'], hits['x2']))
            
    x2 = hits.x2.values
    y2 = hits.y2.values
    z2 = hits.z2.values
    hits['rho'] = np.sqrt(x2**2 + y2**2 + z2**2)

    hits1 = hits[['x2', 'y2', 'z2']]
    hits1.to_csv('../cache/optics_e1000_x2y2z2.csv', index=False, sep=' ')
    hits2 = hits[['phi', 'rho']]
    hits2.to_csv('../cache/optics_e1000_phirho.csv', index=False, sep=' ')
    break #Remove to test more samples
print(sorted(scores, reverse=True))

   hit_id          x         y       z  volume_id  layer_id  module_id  \
0       1 -64.409897 -7.163700 -1502.5          7         2          1   
1       2 -55.336102  0.635342 -1502.5          7         2          1   
2       3 -83.830498 -1.143010 -1502.5          7         2          1   
3       4 -96.109100 -8.241030 -1502.5          7         2          1   
4       5 -62.673599 -9.371200 -1502.5          7         2          1   

   event_id        x2        y2         z2  
0      1000 -0.042829 -0.004763 -23.184208  
1      1000 -0.036804  0.000423 -27.150467  
2      1000 -0.055707 -0.000760 -17.921406  
3      1000 -0.063835 -0.005474 -15.576118  
4      1000 -0.041676 -0.006232 -23.709833  
[]


In [3]:
def template_clustering(path_sample, eps, minpts, amount_clusters = None, visualize = True, ccore = False):
    
    
    optics_instance = optics(sample, eps, minpts, amount_clusters, ccore);
    (ticks, _) = timedcall(optics_instance.process);
    
    print("Sample: ", path_sample, "\t\tExecution time: ", ticks, "\n");
    
    clusters = optics_instance.get_clusters();
    noise = optics_instance.get_noise();
        
    if (visualize is True):
        
    
        visualizer = cluster_visualizer();
        visualizer.append_clusters(clusters, sample);
        visualizer.append_cluster(noise, sample, marker = 'x');
        visualizer.show();
    
        ordering = optics_instance.get_ordering();
        analyser = ordering_analyser(ordering);
        
        ordering_visualizer.show_ordering_diagram(analyser, amount_clusters);
    return clusters, noise

In [4]:
def cluster_x2y2z2(sample, eps, minpts, amount_clusters,visualize):
    return template_clustering(sample, eps, minpts, amount_clusters,visualize);

In [45]:
%%time
clusters, noise = cluster_x2y2z2(0.035, 7, amount_clusters=None, visualize=False);
print(len(clusters))
print(len(noise))
print(len(hits1))

Sample:  ../cache/optics_e1000_x2y2z2.csv 		Execution time:  126.59979199999998 

828
6219
120939
CPU times: user 2min 6s, sys: 8 ms, total: 2min 6s
Wall time: 2min 6s


In [50]:
print(clusters)

[[0, 47, 7, 8015, 3346, 3392, 72, 5463, 5600, 5541, 3290, 1528, 8171, 43, 5531, 5475, 3331, 1505, 1540, 3295, 3231, 8075, 5419, 11031, 1430, 3179, 1461, 5335, 1463, 1490, 3252, 7962, 10700, 5385, 10772, 7898, 7813, 3238], [1, 1459, 1419, 3233, 5342, 3169, 5423, 5481, 7866, 7950, 1251, 1308, 7587, 5109, 7515, 4959, 1207, 2917, 7427, 1157, 2985, 5029, 2883, 10279, 7168, 4848, 7201, 4746, 1013, 4777, 2713, 1029, 7289, 9945, 2684, 7354, 7448, 7305], [3, 3193, 3228, 5325, 13830, 16802, 5402, 7905, 13881, 10848, 7972, 10785, 1427, 10780, 14169, 14040, 1326, 11106, 10977, 8108, 8151, 5544, 184, 11250, 1676, 3508, 3455, 5649, 5695, 11207, 14202, 8333, 14301, 14408, 8234, 8401, 8455, 8603, 8571, 5601, 3347, 3393, 5755, 1583, 1524, 75, 11280, 3064, 5217, 7726, 10625, 10584, 13618, 13552, 16563, 16678, 16635, 13673, 14154, 1316, 11346, 11391, 8444, 8607, 8720, 14376, 5975, 6051, 3816, 3785, 3735, 3788, 1839, 1893, 369, 14290, 14470, 14483, 14319, 14387, 11261, 11416, 220, 11324, 5981, 5900, 14596

In [55]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.01, 0.02, 0.03, 0.04, 0.05]:
            clusters, noise = cluster_x2y2z2(sample, eps, 7, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

Sample:  ../cache/optics_e1000_x2y2z2.csv 		Execution time:  25.281128999999964 

0.01 10566 826 0.05120044777365597
Sample:  ../cache/optics_e1000_x2y2z2.csv 		Execution time:  44.697966999999835 

0.02 10566 3346 0.09693175723625214
Sample:  ../cache/optics_e1000_x2y2z2.csv 		Execution time:  68.95532700000012 

0.03 10566 1939 0.05139695575062514
Sample:  ../cache/optics_e1000_x2y2z2.csv 		Execution time:  209.17366900000002 

0.04 10566 380 0.0133490998196945


TypeError: catching classes that do not inherit from BaseException is not allowed

In [60]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.02, 0.021, 0.022, 0.023, 0.024]:
            clusters, noise = cluster_x2y2z2(sample, eps, 7, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02 10566 3346 0.09693175723625214


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.021 10566 3556 0.09620381875862538


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.022 10566 3667 0.09264607973082409


TypeError: catching classes that do not inherit from BaseException is not allowed

In [61]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0199, 0.020, 0.0201]:
            clusters, noise = cluster_x2y2z2(sample, eps, 7, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0199 10566 3316 0.09684653376637875


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02 10566 3346 0.09693175723625214


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0201 10566 3358 0.09718008699273395
[[0.09718008699273395, 0.0201], [0.09693175723625214, 0.02], [0.09684653376637875, 0.0199]]
CPU times: user 5min 7s, sys: 80 ms, total: 5min 7s
Wall time: 5min 7s


In [62]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0202, 0.0203, 0.0204, 0.0205, 0.0206]:
            clusters, noise = cluster_x2y2z2(sample, eps, 7, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0202 10566 3392 0.096312762078199


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0203 10566 3413 0.09718790938796414


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0204 10566 3443 0.09691922215979334


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0205 10566 3468 0.09773462492865873


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 3498 0.09792870922897773
[[0.09792870922897773, 0.0206], [0.09773462492865873, 0.0205], [0.09718790938796414, 0.0203], [0.09691922215979334, 0.0204], [0.096312762078199, 0.0202]]
CPU times: user 8min 37s, sys: 100 ms, total: 8min 37s
Wall time: 8min 37s


In [64]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0206, 0.02061, 0.02062]:
            clusters, noise = cluster_x2y2z2(sample, eps, 7, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 3498 0.09792870922897773


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02061 10566 3499 0.09755434412301975


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02062 10566 3499 0.09758573992565203
[[0.09792870922897773, 0.0206], [0.09758573992565203, 0.02062], [0.09755434412301975, 0.02061]]
CPU times: user 5min 11s, sys: 56 ms, total: 5min 11s
Wall time: 5min 11s


In [65]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0206, 0.02061, 0.02062]:
            clusters, noise = cluster_x2y2z2(sample, eps, 6, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 4292 0.1059746505622158


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02061 10566 4290 0.10553526917679792


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02062 10566 4290 0.10558732102960736
[[0.1059746505622158, 0.0206], [0.10558732102960736, 0.02062], [0.10553526917679792, 0.02061]]
CPU times: user 5min 13s, sys: 80 ms, total: 5min 13s
Wall time: 5min 13s


In [66]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.02055, 0.0206]:
            clusters, noise = cluster_x2y2z2(sample, eps, 6, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02055 10566 4290 0.10587934167998223


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 4292 0.1059746505622158
[[0.1059746505622158, 0.0206], [0.10587934167998223, 0.02055]]
CPU times: user 3min 29s, sys: 100 ms, total: 3min 29s
Wall time: 3min 29s


In [ ]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0205, 0.0206, 0.0207]:
            clusters, noise = cluster_x2y2z2(sample, eps, 6, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

In [8]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.02055, 0.0206]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02055 10566 5087 0.1077866138679415


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 5075 0.1077167339379447
[[0.1077866138679415, 0.02055], [0.1077167339379447, 0.0206]]
CPU times: user 3min 33s, sys: 170 ms, total: 3min 33s
Wall time: 3min 33s


In [9]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.02055, 0.02050]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02055 10566 5087 0.1077866138679415


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0205 10566 5091 0.10795078246557067
[[0.10795078246557067, 0.0205], [0.1077866138679415, 0.02055]]
CPU times: user 3min 33s, sys: 172 ms, total: 3min 33s
Wall time: 3min 34s


In [10]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0204, 0.0205, 0.0206]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0204 10566 5097 0.10912632231493505


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0205 10566 5091 0.10795078246557067


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0206 10566 5075 0.1077167339379447
[[0.10912632231493505, 0.0204], [0.10795078246557067, 0.0205], [0.1077167339379447, 0.0206]]
CPU times: user 5min 13s, sys: 152 ms, total: 5min 13s
Wall time: 5min 13s


In [11]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0201, 0.0202, 0.0203, 0.0204]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0201 10566 5143 0.1108404230615524


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0202 10566 5127 0.10971456381071554


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0203 10566 5118 0.10919743807504646


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0204 10566 5097 0.10912632231493505
[[0.1108404230615524, 0.0201], [0.10971456381071554, 0.0202], [0.10919743807504646, 0.0203], [0.10912632231493505, 0.0204]]
CPU times: user 6min 56s, sys: 140 ms, total: 6min 56s
Wall time: 6min 56s


In [12]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.01, 0.015, 0.02]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.01 10566 1982 0.08115325018135028


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 4044 0.11853947703353075


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02 10566 5154 0.11122323475899604
[[0.11853947703353075, 0.015], [0.11122323475899604, 0.02], [0.08115325018135028, 0.01]]
CPU times: user 4min 43s, sys: 132 ms, total: 4min 43s
Wall time: 4min 43s


In [13]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.014, 0.015, 0.016]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.014 10566 3600 0.11350300172992567


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 4044 0.11853947703353075


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.016 10566 4482 0.12032657914162218
[[0.12032657914162218, 0.016], [0.11853947703353075, 0.015], [0.11350300172992567, 0.014]]
CPU times: user 4min 40s, sys: 136 ms, total: 4min 40s
Wall time: 4min 40s


In [14]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0165, 0.017, 0.018]:
            clusters, noise = cluster_x2y2z2(sample, eps, 5, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0165 10566 4672 0.11917208872006269


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.017 10566 4836 0.1196959190571231


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.018 10566 5051 0.11848033475437407
[[0.1196959190571231, 0.017], [0.11917208872006269, 0.0165], [0.11848033475437407, 0.018]]
CPU times: user 4min 50s, sys: 124 ms, total: 4min 50s
Wall time: 4min 50s


In [15]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.01, 0.015, 0.02, 0.025, 0.03, 0.035]:
            clusters, noise = cluster_x2y2z2(sample, eps, 4, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.01 10566 3273 0.09831688163153117


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 5972 0.13238702283421472


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

0.025 10566 2984 0.07439139251215629


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.03 10566 1162 0.04187207796701705


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.035 10566 560 0.021785180721702643
[[0.13238702283421472, 0.015], [0.11209300365692533, 0.02], [0.09831688163153117, 0.01], [0.07439139251215629, 0.025], [0.04187207796701705, 0.03], [0.021785180721702643, 0.035]]
CPU times: user 13min 35s, sys: 208 ms, total: 13min 35s
Wall time: 13min 35s


In [16]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.0149, 0.015, 0.0155, 0.016]:
            clusters, noise = cluster_x2y2z2(sample, eps, 4, amount_clusters=None, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0149 10566 5943 0.1323292600328208


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 5972 0.13238702283421472


TypeError: catching classes that do not inherit from BaseException is not allowed

In [18]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.015, 0.0155, 0.016]:
            clusters, noise = cluster_x2y2z2(sample, eps, 4, amount_clusters=10000, visualize=False);
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 5972 0.13238702283421472


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0155 10566 6105 0.13311792406695191


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.016 10566 6258 0.13180975155435942
[[0.13311792406695191, 0.0155], [0.13238702283421472, 0.015], [0.13180975155435942, 0.016]]
CPU times: user 4min 54s, sys: 124 ms, total: 4min 55s
Wall time: 4min 55s


In [22]:
%%time
scores = []
for e in train:
    hits, cells, truth = load_event(e, parts=['hits', 'cells', 'truth'])
    hits['event_id'] = int(e[-9:])
    
    try:
        path_sample = '../cache/optics_e1000_x2y2z2.csv'
        sample = read_sample(path_sample);
        for eps in [0.01, 0.015, 0.0155, 0.02]:
            clusters, noise = cluster_x2y2z2(sample, eps, 3, amount_clusters=10000, visualize=False)
            print(len(clusters), len(noise))
            clusters.append(noise)

            for i, cluster in enumerate(clusters):
                for index in cluster:
                    hits.loc[index, 'track_id'] = i+1

            score = score_event(truth, hits[['event_id','hit_id','track_id']])
            print(eps, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, eps])
    except e:
        print("ERROR:{}".format(e))
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.01 10566 5959 0.11587659832306485


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.015 10566 8575 0.14278726079733672


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.0155 10566 8519 0.1414318774699309


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.02 10566 8152 0.1404662874395841
[[0.14278726079733672, 0.015], [0.1414318774699309, 0.0155], [0.1404662874395841, 0.02], [0.11587659832306485, 0.01]]
CPU times: user 6min 59s, sys: 304 ms, total: 6min 59s
Wall time: 6min 59s


In [21]:
print(len(clusters), len(noise))

8152 61772
